In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from google.colab import drive

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Path untuk Input dan Output
input_path = "/content/drive/MyDrive/Dataset_Telur_Skripsi"
analysis_csv = "/content/drive/MyDrive/Analisis_DataRGB(fix).csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 3. Folder kategori
classes = ['Berkualitas', 'Kurang Berkualitas', 'Tidak Berkualitas']

# Fungsi untuk mendapatkan usia simpan berdasarkan nama file
def get_storage_age(filename):
    if filename.startswith('0_'):
        return '0 minggu'
    elif filename.startswith('1_'):
        return '1 minggu'
    elif filename.startswith('2_'):
        return '2 minggu'
    elif filename.startswith('3_'):
        return '3 minggu'
    elif filename.startswith('4_'):
        return '4 minggu'
    elif filename.startswith('5_'):
        return '5 minggu'
    else:
        return 'Tidak diketahui'

# Fungsi untuk menentukan kelas kualitas berdasarkan usia simpan
def determine_quality_class(filename):
    if filename.startswith('0_') or filename.startswith('1_') or filename.startswith('2_'):
        return 'Berkualitas'
    elif filename.startswith('3_'):
        return 'Kurang Berkualitas'
    else:
        return 'Tidak Berkualitas'

In [ ]:
# 4. Fungsi untuk Crop Gambar Fokus pada Telur Menggunakan Deteksi Kontur
def crop_to_egg(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresholded = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    else:
        return image

In [ ]:
# 5. Fungsi untuk Analisis RGB dan Standar Deviasi
def analyze_rgb_and_std(image):
    mean_rgb = np.mean(image, axis=(0, 1))
    std_rgb = np.std(image, axis=(0, 1))
    return mean_rgb, std_rgb

In [ ]:
# 6. Fungsi untuk Menyimpan Hasil CSV Analisis RGB
def process_and_analyze_rgb(input_path, analysis_csv):
    columns = ['filename', 'mean_R', 'mean_G', 'mean_B', 'std_R', 'std_G', 'std_B', 'usia_simpan', 'kelas_kualitas']
    analysis_df = pd.DataFrame(columns=columns)

    for kls in classes:
        input_dir = os.path.join(input_path, kls)

        for filename in tqdm(os.listdir(input_dir), desc=f"Proses {kls}"):
            if filename.lower().endswith((".jpg", ".png")):  # Periksa format citra lebih umum
                path_img = os.path.join(input_dir, filename)
                img = cv2.imread(path_img)

                # Periksa apakah citra terbaca dengan benar
                if img is None:
                    print(f"Error: Gambar {filename} tidak dapat dibaca.")
                    continue  # Skip jika citra tidak terbaca

                # Crop gambar berdasarkan kontur telur
                cropped_image = crop_to_egg(img)

                # Analisis RGB dan Standar Deviasi setelah cropping
                mean_rgb, std_rgb = analyze_rgb_and_std(cropped_image)

                # Menambahkan analisis RGB dan kolom baru untuk usia simpan dan kelas kualitas ke dataframe
                analysis_row = pd.DataFrame([{
                    'filename': filename,
                    'mean_R': mean_rgb[2],  # RGB order: Red = 2, Green = 1, Blue = 0
                    'mean_G': mean_rgb[1],
                    'mean_B': mean_rgb[0],
                    'std_R': std_rgb[2],
                    'std_G': std_rgb[1],
                    'std_B': std_rgb[0],
                    'usia_simpan': get_storage_age(filename),
                    'kelas_kualitas': determine_quality_class(filename)
                }])

                analysis_df = pd.concat([analysis_df, analysis_row], ignore_index=True)

    # Menyimpan hasil analisis RGB ke CSV
    analysis_df.to_csv(analysis_csv, index=False)

In [ ]:
# 7. Panggil proses untuk analisis RGB dan simpan ke CSV
process_and_analyze_rgb(input_path, analysis_csv)

Proses Berkualitas:   0%|          | 0/270 [00:00<?, ?it/s]/tmp/ipython-input-3961608597.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  analysis_df = pd.concat([analysis_df, analysis_row], ignore_index=True)
Proses Tidak Berkualitas: 100%|██████████| 270/270 [02:38<00:00,  1.71it/s]
